<a href="https://colab.research.google.com/github/Aryanupadhyay23/Deep-Learning-/blob/main/keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd

In [44]:
df = pd.read_csv('diabetes.csv')

In [45]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [46]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [47]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1]

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [49]:
X = scaler.fit_transform(X)

In [50]:
X.shape

(768, 8)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [52]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [53]:
model = Sequential()
model.add(Dense(32,activation='relu', input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
model.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.3356 - loss: 0.8889 - val_accuracy: 0.4156 - val_loss: 0.7757
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4578 - loss: 0.7664 - val_accuracy: 0.5065 - val_loss: 0.7051
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5678 - loss: 0.7023 - val_accuracy: 0.6234 - val_loss: 0.6512
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6296 - loss: 0.6505 - val_accuracy: 0.6883 - val_loss: 0.6114
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6857 - loss: 0.6132 - val_accuracy: 0.7468 - val_loss: 0.5806
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7431 - loss: 0.5778 - val_accuracy: 0.7532 - val_loss: 0.5574
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7425 - loss: 0.5618 - val_accuracy: 0.7468 - val_loss: 0.5384
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7428 - loss: 0.5413 - val_accuracy: 0.7662 

In [55]:
# 1. how to select appropriate optimizer
# 2. number of nodes in a layer
# 3. how to select number of layers
# 4. all in all one model

In [56]:
!pip install keras-tuner

In [57]:
import keras_tuner as kt

In [58]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics = ['accuracy'])

  return model

In [59]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [60]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [61]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7662337422370911

Trial 0 summary
Hyperparameters:
optimizer: adam
Score: 0.7402597665786743

Trial 3 summary
Hyperparameters:
optimizer: sgd
Score: 0.6363636255264282

Trial 1 summary
Hyperparameters:
optimizer: adadelta
Score: 0.48701298236846924


In [62]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [63]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [64]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data =(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7418 - loss: 0.5287 - val_accuracy: 0.7727 - val_loss: 0.5067
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7695 - loss: 0.4940 - val_accuracy: 0.7662 - val_loss: 0.4972
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7910 - loss: 0.4801 - val_accuracy: 0.7792 - val_loss: 0.4897
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7616 - loss: 0.4882 - val_accuracy: 0.7857 - val_loss: 0.4838
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7785 - loss: 0.4849 - val_accuracy: 0.7792 - val_loss: 0.4783
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7824 - loss: 0.4715 - val_accuracy: 0.7857 - val_loss: 0.4762
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7878 - loss: 0.4588 - val_accuracy: 0.7792 - val_loss: 0.4729
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7812 - loss: 0.4644 - val_accuracy: 0.

In [66]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128)

  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=["accuracy"])

  return model

In [67]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir', project_name='neurons')

Reloading Tuner from mydir/neurons/tuner0.json


In [68]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [69]:
tuner.results_summary()

Results summary
Results in mydir/neurons
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units: 83
Score: 0.8051947951316833

Trial 4 summary
Hyperparameters:
units: 124
Score: 0.798701286315918

Trial 1 summary
Hyperparameters:
units: 97
Score: 0.7857142686843872

Trial 0 summary
Hyperparameters:
units: 45
Score: 0.7792207598686218

Trial 3 summary
Hyperparameters:
units: 105
Score: 0.7792207598686218


In [70]:
tuner.get_best_hyperparameters()[0].values

{'units': 83}

In [71]:
model = tuner.get_best_models(num_models=1)[0]

In [72]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data =(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7586 - loss: 0.4783 - val_accuracy: 0.7922 - val_loss: 0.4651
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7668 - loss: 0.4612 - val_accuracy: 0.7857 - val_loss: 0.4621
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7701 - loss: 0.4573 - val_accuracy: 0.7922 - val_loss: 0.4603
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7658 - loss: 0.4393 - val_accuracy: 0.7857 - val_loss: 0.4600
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7605 - loss: 0.4673 - val_accuracy: 0.7922 - val_loss: 0.4587
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7823 - loss: 0.4393 - val_accuracy: 0.7922 - val_loss: 0.4594
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7771 - loss: 0.4531 - val_accuracy: 0.7857 - val_loss: 0.4589
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7715 - loss: 0.4543 - val_accuracy: 0.79

In [73]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(102, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(102, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [74]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', directory='mydir',
                      project_name='num_layers')

Reloading Tuner from mydir/num_layers/tuner0.json


In [75]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))


In [76]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [77]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [78]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7766 - loss: 0.4595 - val_accuracy: 0.8117 - val_loss: 0.4793
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7937 - loss: 0.4624 - val_accuracy: 0.7987 - val_loss: 0.5189
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7904 - loss: 0.4285 - val_accuracy: 0.7727 - val_loss: 0.5164
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7935 - loss: 0.4310 - val_accuracy: 0.7987 - val_loss: 0.5014
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8019 - loss: 0.3930 - val_accuracy: 0.8052 - val_loss: 0.5179
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8181 - loss: 0.3781 - val_accuracy: 0.7792 - val_loss: 0.5020
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8356 - loss: 0.3750 - val_accuracy: 0.7857 - val_loss: 0.5187
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8338 - loss: 0.3685 - val_accuracy: 0.80

In [79]:
def build_model(hp):
  model = Sequential()

  num_layers = hp.Int('num_layers', min_value=1, max_value=10)

  model.add(Dense(
      units=hp.Int('units_0', min_value=8, max_value=128, step=8),
      activation=hp.Choice('activation_0', values=['relu', 'tanh', 'sigmoid']),
      input_dim=8
  ))
  model.add(Dropout(hp.Choice('dropout_0',values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

  for i in range(1, num_layers):
    model.add(Dense(
        units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
        activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid'])
    ))
    model.add(Dropout(hp.Choice('dropout_' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [80]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final')

Reloading Tuner from mydir/final/tuner0.json


In [81]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

In [82]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units_0': 64,
 'activation_0': 'tanh',
 'dropout_0': 0.2,
 'optimizer': 'rmsprop',
 'units_1': 80,
 'activation_1': 'relu',
 'dropout_1': 0.2,
 'units_2': 96,
 'activation_2': 'relu',
 'dropout_2': 0.5,
 'units_3': 64,
 'activation_3': 'sigmoid',
 'dropout_3': 0.5,
 'units_4': 120,
 'activation_4': 'relu',
 'dropout_4': 0.8,
 'units_5': 56,
 'activation_5': 'sigmoid',
 'dropout_5': 0.3,
 'units_6': 88,
 'activation_6': 'tanh',
 'dropout_6': 0.2,
 'units_7': 72,
 'activation_7': 'sigmoid',
 'dropout_7': 0.8,
 'units_8': 72,
 'activation_8': 'relu',
 'dropout_8': 0.6,
 'units_9': 72,
 'activation_9': 'sigmoid',
 'dropout_9': 0.2}

In [83]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [84]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6126 - loss: 0.6938 - val_accuracy: 0.6429 - val_loss: 0.6542
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6429 - loss: 0.6647 - val_accuracy: 0.6429 - val_loss: 0.6797
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6157 - loss: 0.6723 - val_accuracy: 0.6429 - val_loss: 0.6571
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6504 - loss: 0.6534 - val_accuracy: 0.6429 - val_loss: 0.6582
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6122 - loss: 0.6815 - val_accuracy: 0.6429 - val_loss: 0.6918
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6433 - loss: 0.6690 - val_accuracy: 0.6429 - val_loss: 0.6471
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6243 - loss: 0.6796 - val_accuracy: 0.6429 - val_loss: 0.6374
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6353 - loss: 0.6430 - val_accuracy: 0.64